# [5-6] colorTracking

이 문서는 기본적으로 밝은 노란색 객체를 추적하는 색상 추적 기능을 구현하는 데 사용됩니다.  
참고: 이 문서에는 서보 제어가 포함되어 있습니다.  
코드를 실행할 때 로봇 팔의 움직임 범위에 부서지기 쉬운 물체가 있는지 주의하세요.  또한 어린이로부터 멀리 떨어뜨려야 합니다.


## Import camera function libraries

다음 코드 블록을 실행한 후에는 잠시 기다렸다가 카메라가 초기화되기를 기다리십시오.  
초기화가 성공하면 300x300 크기의 실시간 비디오 화면이 코드 블록 아래에 나타납니다.  

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Importing the TTLServo class

In [2]:
from SCSCtrl import TTLServo
import time

# TTL 서보가 너무 자주 통신하는 경우에는 일정한 확률로 rx 및 tx 통신 오류가 발생할 수 있습니다.
# 각 통신 후에 일정 시간 동안 지연을 정의하여 과도한 통신 주파수를 피합니다.
servoCtrlTime = 0.001

# 1번과 5번 서보를 초기 위치로 돌립니다.
# 1번 서보는 카메라 팬/틸트의 PAN 축 이동을 제어하여 수평으로 회전합니다.
# 5번 서보는 카메라 팬/틸트의 TILT 축 이동을 제어하여 수직 방향으로 위아래로 기울입니다.
TTLServo.servoAngleCtrl(1, 0, 1, 150)
time.sleep(servoCtrlTime)
TTLServo.servoAngleCtrl(5, 0, 1, 150)
time.sleep(servoCtrlTime)

# 카메라 방향 함수
def cameraUp(speedInput):
    TTLServo.servoAngleCtrl(5, -70, 1, speedInput)
    time.sleep(servoCtrlTime)
def cameraDown(speedInput):
    TTLServo.servoAngleCtrl(5, 25, 1, speedInput)
    time.sleep(servoCtrlTime)

def ptRight(speedInput):
    TTLServo.servoAngleCtrl(1, 80, 1, speedInput)
    time.sleep(servoCtrlTime)

def ptLeft(speedInput):
    TTLServo.servoAngleCtrl(1, -80, 1, speedInput)
    time.sleep(servoCtrlTime)

# 카메라의 틸트 축 이동이 멈춥니다.
def tiltStop():
    TTLServo.servoStop(5)
    time.sleep(servoCtrlTime)

# 카메라의 팬 축 이동이 멈춥니다.
def panStop():
    TTLServo.servoStop(1)
    time.sleep(servoCtrlTime)

# 위의 코드 블록을 실행한 후에는 카메라 팬/틸트의 1번과 5번 서보가 천천히 중간 위치로 회전합니다.

Succeeded to open the port
Succeeded to change the baudrate


## Color recognition and tracking function

`ColorRecognition` 챕터에서, 객체 색상의 HSV 값을 구하는 방법을 알아보았다.  
`colorUpper ` 에 최대 값, `colorLower` 에 최소 값을 주고, `np.array([H, S, V])` 형식에 주의하자.

자신의 객체 색상으로 대체하지 않는 경우 기본 프로그램을 사용하여 색상 인식을 구현할 수도 있습니다.  
기본 추적 색상은 밝은 노란색 객체입니다.

In [5]:
import cv2
import numpy as np

# 색상 HSV 샘플

#Yellow #FFFF00
colorUpper = np.array([44, 255, 255])
colorLower = np.array([24, 100, 100])

# Red FF0000
# colorUpper = np.array([180, 255, 255])
# colorLower = np.array([160, 100, 100])

# Green #00FF00
# colorUpper = np.array([50, 255, 255])
# colorLower = np.array([70, 200, 100])

# Blue #0000FF
# colorUpper = np.array([110, 225, 255])
# colorLower = np.array([135, 180, 200])

# Cyan #00FFFF
# colorUpper = np.array([80, 255, 255])
# colorLower = np.array([105, 180, 180])

# Magenta #FF00FF
# colorUpper = np.array([140, 255, 255])
# colorLower = np.array([160, 150, 200])

# 카메라가 이 객체를 향해 회전할 때의 위치 허용 오차를 정의합니다.
# 값이 높을수록 카메라의 조준 정확도가 높아지지만, 
# 너무 높은 값은 카메라가 지속적으로 흔들리는 것을 초래할 수도 있습니다.
error_tor = 25

# 이것은 간단한 PID 조절기의 P 값입니다.
# 이 값은 움직임 속도의 비례적 조절 계수입니다.
# 이 값이 너무 높으면 카메라 PT 움직임이 과도하게 진동할 수 있고,
# 너무 낮으면 색상 추적 응답 속도가 너무 느려질 수 있습니다.

PID_P = 3

# 색상 인식 후 추적하는 함수
def findColor(imageInput):
    # HSV로 변환
    hsv = cv2.cvtColor(imageInput, cv2.COLOR_BGR2HSV)
    
    # 대상 색상과 일치하는 픽셀에 대한 마스크를 생성합니다.
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    
    # 침식(Erode)은 선택된 마스크에서 상대적으로 작은 영역을 제거하는 과정입니다. 
    # 이는 노이즈 제거로 이해할 수 있습니다.    
    mask = cv2.erode(mask, None, iterations=2)
    
    # 팽창(dilate)은 이전의 침식 과정으로 인해 큰 영역이 작아지고 작은 영역이 
    # 사라졌을 것입니다. 
    # 이 단계는 큰 영역을 이전 크기로 복원하는 것입니다.    
    mask = cv2.dilate(mask, None, iterations=2)
    
    # 적합한 영역의 윤곽선을 얻습니다.
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None
    
    # 만일 일치하는 영역이 있다면, 색상 추적을 달성하기 위해 조향 장치의 움직임을 제어하기 시작합니다.
    if len(cnts) > 0:
        # "대상을 찾았음"을 나타내기 위해 텍스트를 그립니다.
        imageInput = cv2.putText(imageInput,'Target Detected',(10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
        
        # 가장 큰 영역의 윤곽을 찾습니다.
        c = max(cnts, key=cv2.contourArea)
        
        # 이 영역의 중심점 위치와 이 영역의 반경을 구합니다.
        ((box_x, box_y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        
        # X와 Y는 해당 영역의 중심점입니다.
        X = int(box_x)
        Y = int(box_y)
        
        # error_X, error_Y는 해당 영역의 중심점과 프레임의 중심점 간의 오차의 절대값입니다.
        error_Y = abs(150 - Y)
        error_X = abs(150 - X)
        
        # 이 영역의 크기와 위치를 그립니다.
        cv2.rectangle(imageInput,(int(box_x-radius),int(box_y+radius)),(int(box_x+radius),int(box_y-radius)),(255,255,255),1)
        
        if Y < 150 - error_tor:
            # 카메라 위로 향하기
            imageInput = cv2.putText(imageInput,'Looking Up',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            cameraUp(error_Y*PID_P)
        elif Y > 150 + error_tor:
            # 카메라 아래로 향하기
            imageInput = cv2.putText(imageInput,'Looking Down',(10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            cameraDown(error_Y*PID_P)
        else:
            # 수직 방향의 오차가 허용 오차보다 작으면, 
            # 카메라는 피치 방향으로의 이동을 멈춥니다.
            imageInput = cv2.putText(imageInput,'Y Axis Locked',(10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            tiltStop()

        if X < 150 - error_tor:
            # 카메라 왼쪽 바라보기
            imageInput = cv2.putText(imageInput,'Looking Left',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            ptLeft(error_X*PID_P)
        elif X > 150 + error_tor:
            # 카메라 오른쪽 바라보기
            imageInput = cv2.putText(imageInput,'Looking Right',(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            ptRight(error_X*PID_P)
        else:
            # 수평 방향의 오차가 허용 오차보다 작으면,
            # 카메라는 수평 방향으로의 이동을 멈춥니다.
            imageInput = cv2.putText(imageInput,'X Axis Locked',(10,80), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
            panStop()

    # 대상 색상과 일치하는 영역을 찾지 못하면, 카메라는 회전을 멈춥니다.
    else:
        imageInput = cv2.putText(imageInput,'Target Detecting',(10,20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
        tiltStop()
        panStop()
    
    return imageInput

다음 코드를 실행한 후에는 보이는 프레임이 변경되지 않습니다.  
실제 효과를 관찰하려면 image_widget.value의 값이 변경되고 camera.observe() 함수가 호출된 후에야 합니다.  
전자의 값은 처리된 프레임을 표시하는 데 사용되고, 후자 함수는 새 프레임이 수집된 직후 이미지 처리 관련 메서드를 호출하는 데 사용됩니다.  

다음 코드 블록을 실행하여 색상 인식 및 추적 기능을 활성화합니다.   
참고: 이 문서에는 서보 제어가 포함되어 있습니다.  
코드를 실행할 때 로봇 팔의 움직임 범위에 부서지기 쉬운 물체가 있는지 주의하세요. 또한 어린이로부터 멀리 떨어뜨려야 합니다.


In [6]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(findColor(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

[TxRxResult] Incorrect status packet!


## Turn off this function processing and stop the camera
이 함수 처리를 중지하고 카메라를 종료하세요.

In [7]:
camera.unobserve(execute, names='value')

time.sleep(1)

tiltStop()
panStop()
camera.stop()